<h2 style="color:red">Imports</h2>

In [2]:
#imports 
#from tensorflow import keras
import keras
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import LSTM,Dense,Input,Bidirectional
from nltk.tokenize.treebank import TreebankWordTokenizer
from scipy import spatial
from random import shuffle
import pickle
from keras.callbacks import ModelCheckpoint
from scipy import spatial
import csv

Using TensorFlow backend.


<h2 style="color:red">Word Embedding</h2>

In [13]:
embeddings_index = {}
embeddings_size = 300
i = 0
with open('/Users/mohammad/Documents/Internship-IAI/farsi_testing/cc.fa.300.vec') as f:
    for line in f:
        if i > 400000:
            break
        i += 1
        values = line.split()
        word = values[0]
        coefs = [float(i) for i in values[1:]]
        embeddings_index[word] = coefs
embeddings_index['<PAD>'] = [0] * 300
embeddings_index['<UNK>'] = [1] * 300

In [14]:
def sim(dataSetI , dataSetII):
    return 1 - spatial.distance.cosine(dataSetI, dataSetII)

<h2 style="color:red">Preparing our Data</h2>

In [15]:
marks = {'؟', '!', '.', '،'}
zamir_1 = {'م' , 'ش', 'ت', 'ه','و','ی'} 
zamir_3 = {'تان', 'شان', 'مان'}
def prepareSent(sent):
    sent = sent.split()
    sent = [i[:-1] if i[-1] in marks or (i[-1] in zamir_1 and i[:-1] in embeddings_index and i not in embeddings_index) else i for i in sent]
    sent = [i[:-3] if i[-3:] in zamir_3 and i[:-3] in embeddings_index and i not in embeddings_index else i for i in sent]
    tokenized = []
    n = 0
    while(n != len(sent)):
        if n < len(sent) - 2 and (sent[n] + "‌" + sent[n+1] + "‌" + sent[n+2]) in embeddings_index:
            tokenized.append(sent[n] + "‌" + sent[n+1] + "‌" + sent[n+2])
            n += 2
        elif n != len(sent) -1 and (sent[n] + "‌" + sent[n+1]) in embeddings_index:
            tokenized.append(sent[n] + "‌" + sent[n+1])
            n += 1
        else:
            tokenized.append(sent[n])
        n += 1
    return tokenized

In [34]:
class_size = 4
X_train = []
Y_train = []
for i in range(1,class_size+1):
    with open('class{}.csv'.format(i)) as f:
        reader = csv.reader(f)
        class_list = [r[0] for r in reader]
        y = [1 if j == i else 0 for j in range(1 , class_size+1)]
        class_list = [(o,y) for o in class_list]
        X_train = X_train + class_list

In [35]:
def whichClass(inp):
    for i in range(len(inp)):
        if 1 == inp[i]:
            if i == 0:
                return "Address"
            elif i == 1:
                return "Resturant"
            elif i == 2:
                return "Home Service"
            elif i == 3:
                return "Laundary"

In [36]:
shuffle(X_train)
shuffle(X_train)
shuffle(X_train)

In [37]:
Y_train = [i[1] for i in X_train]
X_train = [prepareSent(i[0]) for i in X_train]

In [38]:
MAX_SEQ = 20
def wordToVec(data):
    for s in range(len(data)):
        n = MAX_SEQ - len(data[s])
        if n < 0:
            data[s] = data[s][:MAX_SEQ]
        else:
            for i in range(n):
                data[s].append('<PAD>')
        for v in range(len(data[s])):
            if data[s][v] not in embeddings_index:
                data[s][v] = embeddings_index['<UNK>']
            else:
                data[s][v] = embeddings_index[data[s][v]]
    return np.array(data)

In [39]:
X_train = wordToVec(X_train)
X_train.shape

(403, 20, 300)

<h2 style="color:red">Building model with Batch size 64</h2>


In [40]:
BATCH_SIZE = 16
input_layer = Input( batch_shape = (BATCH_SIZE, MAX_SEQ, 300))
lstm_layer = Bidirectional(LSTM(units=MAX_SEQ))(input_layer)
output_layer = Dense(class_size, activation="softmax")(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy',
              optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (16, 20, 300)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (16, 40)                  51360     
_________________________________________________________________
dense_2 (Dense)              (16, 4)                   164       
Total params: 51,524
Trainable params: 51,524
Non-trainable params: 0
_________________________________________________________________


In [46]:
X_train = X_train[:400]
Y_train = np.array(Y_train[:400])

In [47]:
checkpoint = ModelCheckpoint('weight_farsi_dekhte.{epoch:02d}.hdf5')
callbacks_list = [checkpoint]

In [49]:
#fit the model
EPOCH_SIZE = 50
model.fit(X_train, Y_train, epochs=EPOCH_SIZE, batch_size=BATCH_SIZE, callbacks=callbacks_list)

Epoch 1/50
400/400 [==============================] - 0s 953us/step - loss: 0.0013
Epoch 2/50
400/400 [==============================] - 0s 913us/step - loss: 0.0013
Epoch 3/50
400/400 [==============================] - 0s 894us/step - loss: 0.0012
Epoch 4/50
400/400 [==============================] - 0s 886us/step - loss: 0.0012
Epoch 5/50
400/400 [==============================] - 0s 936us/step - loss: 0.0013
Epoch 6/50
400/400 [==============================] - 0s 913us/step - loss: 0.0011
Epoch 7/50
400/400 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 8/50
400/400 [==============================] - 0s 1ms/step - loss: 9.8185e-04
Epoch 9/50
400/400 [==============================] - 1s 2ms/step - loss: 9.8225e-04
Epoch 10/50
400/400 [==============================] - 0s 1ms/step - loss: 9.2284e-04
Epoch 11/50
400/400 [==============================] - 0s 1ms/step - loss: 8.8754e-04
Epoch 12/50
400/400 [==============================] - 0s 971us/step - loss: 8.

In [52]:
BATCH_SIZE = 1
input_layer = Input( batch_shape = (BATCH_SIZE, MAX_SEQ, 300))
lstm_layer = Bidirectional(LSTM(units=MAX_SEQ))(input_layer)
output_layer = Dense(class_size, activation="softmax")(lstm_layer)

dfmodel = Model(inputs=input_layer, outputs=output_layer)
dfmodel.compile(loss='categorical_crossentropy',
              optimizer='adam')
dfmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (1, 20, 300)              0         
_________________________________________________________________
bidirectional_4 (Bidirection (1, 40)                   51360     
_________________________________________________________________
dense_4 (Dense)              (1, 4)                    164       
Total params: 51,524
Trainable params: 51,524
Non-trainable params: 0
_________________________________________________________________


In [53]:
we = model.get_weights()
dfmodel.set_weights(we)

In [54]:
#fit model
EPOCH_SIZE = 20
dfmodel.fit(X_train, Y_train, epochs=EPOCH_SIZE, batch_size=BATCH_SIZE, callbacks=callbacks_list)

Epoch 1/20
400/400 [==============================] - 5s 14ms/step - loss: 0.1632
Epoch 2/20
400/400 [==============================] - 4s 11ms/step - loss: 0.0743
Epoch 3/20
400/400 [==============================] - 4s 11ms/step - loss: 0.0283
Epoch 4/20
400/400 [==============================] - 5s 12ms/step - loss: 0.0061
Epoch 5/20
400/400 [==============================] - 4s 11ms/step - loss: 0.0035
Epoch 6/20
400/400 [==============================] - 5s 12ms/step - loss: 0.0026
Epoch 7/20
400/400 [==============================] - 5s 12ms/step - loss: 0.0019
Epoch 8/20
400/400 [==============================] - 5s 13ms/step - loss: 0.0015
Epoch 9/20
400/400 [==============================] - 5s 13ms/step - loss: 0.0011
Epoch 10/20
400/400 [==============================] - 5s 12ms/step - loss: 8.6719e-04
Epoch 11/20
400/400 [==============================] - 5s 12ms/step - loss: 6.6353e-04
Epoch 12/20
400/400 [==============================] - 5s 12ms/step - loss: 5.6430e-04
E

In [191]:
def classify(sent):
    sentence = prepareSent(sent)
    sentence = wordToVec([sentence])[0]
    sentence = np.reshape(sentence , (1 , 20 , 300))
    
    sentence = dfmodel.predict(sentence)
    argmax = np.argmax(sentence)
    print(sentence)
    a = [1 if argmax == i else 0 for i in range(class_size)]
    print("In class: " , whichClass(list(a)))

In [5]:
entity_lists = { "cloth" : ['شلوار' , 'کاپشن' , 'پیراهن'],
                "city_iran" : ['شیراز', 'کرج', 'مشهد'],
                "food" : ['کباب', 'ساندویچ', 'سوپ'],
                "time" : ['امروز', 'فردا', 'دیروز']
               }

In [3]:
#similarity function
def sim(dataSetI , dataSetII):
    return 1 - spatial.distance.cosine(dataSetI, dataSetII)

In [45]:
sent = "میدونی چجوری باید برم سی و سه پل"
# sent = 'میخوام برم نظر'
cl = classify(sent)
sent = sent.replace(' ','+')
print(cl)
if cl == 'Address':
    res = requests.get("https://www.google.com/maps/search/?api=1&query=اصفهان+{}&hl=fa".format(sent))
    print(res.url)

Address
https://www.google.com/maps/search/?api=1&query=%D8%A7%D8%B5%D9%81%D9%87%D8%A7%D9%86+%D9%85%DB%8C%D8%AF%D9%88%D9%86%DB%8C+%DA%86%D8%AC%D9%88%D8%B1%DB%8C+%D8%A8%D8%A7%DB%8C%D8%AF+%D8%A8%D8%B1%D9%85+%D8%B3%DB%8C+%D9%88+%D8%B3%D9%87+%D9%BE%D9%84&hl=fa
